![](https://gmufourthestate.com/files/2020/02/NFL-Summary_FERGUSON_2.10.20_RGB-768x374.jpg)

This is the first time that I have tried to create an animation in matplotlib. Although the result looks like a tv video game from the 90's, I'm quite happy that I was able to replicate the animation from the [tutorial](https://www.kaggle.com/tombliss/tutorial) written in R.

After reading many articles, I found this [blog](https://brushingupscience.com/2016/06/21/matplotlib-animations-the-easy-way/) to be the best material to understand matplotlib animations. Similar steps from the blog has been followed here:
- Choose a random game and play from that game.
- Create a scatter plot with jersey numbers.
- Animate the plot with FuncAnimation method with new xy position offsets from consecutive frame ids.

I will be using a random football image as plot background.

### Imports and reading data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation


In [ ]:
plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
week_1 = pd.read_csv('../input/nfl-big-data-bowl-2021/week1.csv')

### Football field image for plot background

In [ ]:
football_field = mpimg.imread('../input/nfl2021/images/football_field.jpg')

### Choosing a random game and play
The main idea here is to choose a random game and play from week1 tracking data and filter down to frameId 1. This data will be used for the scatter plot. Variables - x, y, c and s are used for styling the scatter plot markers. Jersey numbers are needed to annotate the markers and later used in the animation.

In [ ]:
#create data for randomly chosen game and play
random_game = np.random.choice(week_1['gameId'].unique())
data = week_1[week_1['gameId'] == random_game]
random_play = np.random.choice(data['playId'].unique())
data = data[data['playId']==random_play]

#x, y, marker color, marker sizes and jersey num for creating scatter plot for the first frame
cond = data['frameId'] == 1
colors = {'home':'whitesmoke', 'away':'coral', 'football':'brown'}
size = {'home':300, 'away':300, 'football':50}

x = data.loc[cond, 'x']
y = data.loc[cond, 'y']
c = data.loc[cond, 'team'].map(colors)
s = data.loc[cond, 'team'].map(size)
jersey_num = data.loc[cond, 'jerseyNumber'].values
jersey_num = ['' if np.isnan(num) else str(int(num)) for num in jersey_num] #assigning empty string for nan value from football jersey number
zipped = list(zip(x, y, jersey_num)) #for looping and creating annotations on the marker

### Creating the plot
We can now create the scatter plot with annotations with the football field as image background. They are saved to "scatter" and "annotations" variables respectively. Below is a summary of the code:
- The plot dimensions are set to 0, 120, 0, 53.3 as given in the [data overview](https://www.kaggle.com/c/nfl-big-data-bowl-2021/data).
- xmin and xmax are used for zooming into the plot
- xmean is used for x position in plot_title. A simple ax.set_title works but it looks very bad in the animation, hence the complicated code with ax.text and creating two line breaks to keep the title text within the plot.

In [ ]:
#set up the plot
fig, ax = plt.subplots(figsize=(14,8))
imgplot = ax.imshow(football_field, extent=[0, 120, 0, 53.3])

#cap xmin at 0 and xman at 120
xmin = 0 if ((data['x'].min()-20) < 0 or (data['x'].min() < 0)) else data['x'].min()-20
xmax = 120 if ((data['x'].max()+20 > 120) or (data['x'].max() > 120)) else data['x'].max()+20
#xmean for centering title text
xmean = np.mean([xmin, xmax])
ax.set(xticks=([]), yticks=([]), xlim=(xmin, xmax), ylim=(-5, 53.3));

#creating title with two line breaks to ensure title stays within plot
playId = data['playId'].unique().item() #retrieve playId from data
gameId = data['gameId'].unique().item() #retrieve gameId from data
title_cond = (plays['playId'] == playId) & (plays['gameId'] == gameId) #use playId and gameId as filter condition

title_list = plays.loc[title_cond, 'playDescription'].values.item().split(' ') #save title to list
insert_index = int(round(len(title_list)/3)) #index position for inserting two line breaks

for i in [insert_index, insert_index*2]: #insert two line break
    title_list.insert(i, '\n')
    
title = ' '.join(title_list) #join title list on ' '
plot_title = ax.text(x=xmean, y=-2.5, s=title, fontsize=11, ha='center', va='center') #plot title with two line break

#setting up the legend with white text
team = ['home', 'away']
color = list(colors.values())[:2]
team_color = list(zip(team, color))
for team, color in team_color:
    ax.scatter([], [], label=team, c=color)

legend = ax.legend(frameon=False)

#plot scatter points and annotations
scatter = ax.scatter(x, y, marker='o', linestyle='None', c=c, s=s, edgecolor='black', linewidths=1.2,)
annotations = [ax.annotate(val[2], xy=(val[:2]), va='center', ha='center') for val in zipped]

### Animating the plot

The FuncAnimation function needs three main arguments for the animation to work, the figure object on which to animate, frames argument for the number of frames in the animation and a function that will be called repeatedly with number of frames. The interval argument is the time in milliseconds to wait before showing the next frame, this argument is optional and used to control the speed of the animation.

The animate function created below gets the new xy postions for the scatter plot marker and annotations based on the frame_id. frames argument from FuncAnimation is passed to frame_id, so we don't have pass anything to the animate function.

In [ ]:
#create animate function to update scatter plot and annotations
def animate(frame_id):
    #set new xy postions for previously created scatter plot
    xy = data.loc[data['frameId']==frame_id, ['x', 'y']].to_numpy()
    scatter.set_offsets(xy)
    
    #set new xy postions for previously created annotations
    if frame_id > 0:
        [annotations[i].set_position(tuple(xy[i])) for i in range(len(xy))]

        
#call funcanimation
anim = FuncAnimation(fig, animate, interval=100, frames=len(data['frameId'].unique())+1)
plt.rcParams['animation.html'] = 'html5'
anim

### Conclusion
As suggested earlier this [blog](https://brushingupscience.com/2016/06/21/matplotlib-animations-the-easy-way/) post is probably the best place to easily understand matplotlib animation. Also this [youtube](https://www.youtube.com/watch?v=F57_0XPdhD8&t=126s) video might help. Thank you for taking the time to read the notebook. Please upvote if you liked it.